In [1]:
import pandas as pd

# Step 1: Load the Excel file
file_path = "/kaggle/input/gse18520/GSE18520.xlsx"  # Replace with your actual file path
data = pd.read_excel(file_path)

# Step 2: Save the data to a CSV file
csv_file_path = "GSE18520.csv"  # Specify the output CSV file path
data.to_csv(csv_file_path, index=False)  # Save without the index

print(f"File converted to CSV and saved as {csv_file_path}")


File converted to CSV and saved as GSE18520.csv


In [14]:
import pandas as pd

# Step 1: Load the CSV file
file_path = "/kaggle/working/GSE18520.csv"  # Replace with your file path
data = pd.read_csv(file_path)

# Step 2: Keep only the specified columns
columns_to_keep = ['ID', 'adj.P.Val', 'P.Value', 'logFC', 'Gene.symbol']
filtered_data = data[columns_to_keep]

# Step 3: Filter the data based on conditions
# Upregulated genes
upregulated_gene = filtered_data[(filtered_data['adj.P.Val'] < 0.05) & (filtered_data['logFC'] > 1)]

# Downregulated genes
downregulated_gene = filtered_data[(filtered_data['adj.P.Val'] < 0.05) & (filtered_data['logFC'] < -1)]

# All data (already filtered to include only the specified columns)
all_data = filtered_data

# Step 4: Save the filtered data
upregulated_gene.to_csv('GSE18520_upregulated_gene.csv', index=False)
downregulated_gene.to_csv('GSE18520_downregulated_gene.csv', index=False)
all_data.to_csv('GSE18520_all_genes.csv', index=False)

print("Files created successfully!")


Files created successfully!


In [17]:
import pandas as pd

# Load the CSV file
file_path = '/kaggle/working/GSE18520_upregulated_gene.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Check column names
print(df.columns)

# Ensure 'Gene.symbol' is correctly referenced, handle potential whitespace
df.columns = df.columns.str.strip()  # Strip any whitespace from column names

# Filter out rows where 'Gene.symbol' is null, 'nan', or 'NAN', if 'Gene.symbol' exists
if 'Gene.symbol' in df.columns:
    # Remove rows with NaN or 'nan'/'NAN' in 'Gene.symbol'
    filtered_df = df[df['Gene.symbol'].notna() & ~(df['Gene.symbol'].str.upper() == 'NAN')]

    # Ensure 'P.Value' column exists and remove duplicates based on 'Gene.symbol', keeping the highest 'P.Value'
    if 'P.Value' in filtered_df.columns:  # Replace 'P.Value' with the actual p-value column name if different
        filtered_df = filtered_df.loc[filtered_df.groupby('Gene.symbol')['P.Value'].idxmax()]
        
        # Save the filtered DataFrame to a new CSV file
        filtered_df.to_csv('filtered_upregulated_GSE18520.csv', index=False)
        print("Rows with null, 'NAN' or 'nan' values in 'Gene.symbol' have been filtered out, and duplicates were removed based on the highest p-value.")
    else:
        print("Column 'P.Value' not found in the CSV file.")
else:
    print("Column 'Gene.symbol' not found in the CSV file.")


Index(['ID', 'adj.P.Val', 'P.Value', 'logFC', 'Gene.symbol'], dtype='object')
Rows with null, 'NAN' or 'nan' values in 'Gene.symbol' have been filtered out, and duplicates were removed based on the highest p-value.


In [2]:
import pandas as pd

# Define input file names
file1 = "/kaggle/input/filtered-deg/filtered_downregulated_GSE26712.csv"  # Change to your actual file name
file2 = "/kaggle/input/filtered-deg/filtered_upregulated_GSE26712.csv"

# Define output file name
output_file = "combined_DEG_Filtered_GSE26712.csv"

# Read the CSV files
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Combine the two dataframes by stacking rows
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined data to a new CSV file
combined_df.to_csv(output_file, index=False)

print(f"Combined CSV saved as {output_file}")


Combined CSV saved as combined_DEG_Filtered_GSE26712.csv


In [4]:
import pandas as pd
import numpy as np

# Load CSV file
input_file = "/kaggle/working/combined_DEG_Filtered_GSE26712.csv"  # Change this to your CSV file path
output_file = "combined_DEG_Filtered_GSE26712_log2.csv"

# Read the CSV file
df = pd.read_csv(input_file)

# Apply log2 transformation to numeric columns (avoiding non-numeric ones)
df_log2 = df.applymap(lambda x: np.log2(x) if np.issubdtype(type(x), np.number) and x > 0 else x)

# Save transformed data to a new CSV file
df_log2.to_csv(output_file, index=False)

print(f"Log2 transformed data saved to {output_file}")


Log2 transformed data saved to combined_DEG_Filtered_GSE26712_log2.csv


<ipython-input-4-cbd906c0fdc1>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_log2 = df.applymap(lambda x: np.log2(x) if np.issubdtype(type(x), np.number) and x > 0 else x)


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import quantile_transform
import os

# Step 1: Specify the input and output file paths
input_file = '/kaggle/working/combined_DEG_Filtered_GSE18520.csv'  # Replace with your input CSV file
output_file = 'GSE18520_log_quantile_normalized_data.csv'  # Output CSV file

# Step 2: Check if the input file exists
if not os.path.exists(input_file):
    raise FileNotFoundError(f"Input file '{input_file}' not found. Please check the path.")

# Step 3: Read the microarray dataset from the CSV file
microarray_data = pd.read_csv(input_file)

# Ensure the "ID_REF" column exists
if "ID_REF" not in microarray_data.columns:
    raise ValueError("'ID_REF' column not found in the dataset. Ensure the dataset has this column.")

# Set the "ID_REF" column as the index
microarray_data.set_index("ID_REF", inplace=True)

# Step 4: Handle non-positive values before log2 transformation
if (microarray_data <= 0).any().any():
    print("Warning: Dataset contains non-positive values. Adding a small constant (1e-5) to the data.")
    microarray_data += 1e-5  # Make all values strictly positive

# Apply log2 transformation
log2_transformed_data = np.log2(microarray_data)

# Step 5: Perform quantile normalization on the log2-transformed data
quantile_normalized_data = quantile_transform(
    log2_transformed_data, 
    n_quantiles=min(50, log2_transformed_data.shape[0]),  # Adjust n_quantiles to data size
    axis=0, 
    random_state=0, 
    copy=True
)

# Step 6: Convert the normalized data back to a DataFrame
quantile_normalized_df = pd.DataFrame(
    quantile_normalized_data, 
    columns=microarray_data.columns, 
    index=microarray_data.index
)

# Step 7: Write the normalized data to a new CSV file
quantile_normalized_df.to_csv(output_file)

print(f"Normalized data has been saved to '{output_file}'")


ValueError: 'ID_REF' column not found in the dataset. Ensure the dataset has this column.

In [2]:
import pandas as pd

# Step 1: Load the CSV file
input_file = '/kaggle/input/1852000/merged_GSE18520_fresh.csv'  # Replace with your actual file name
output_file = 'merged_GSE18520_fresh_with_Class.csv'

df = pd.read_csv(input_file, index_col="ID_REF")  # Set ID_REF as index

# Step 2: Create the "Class" row
class_values = []  # List to store class values

for column in df.columns:
    if 'GSM462643' <= column <= 'GSM462652':  # Check if column falls in range
        class_values.append(0)
    else:
        class_values.append(1)

# Step 3: Append the new "Class" row at the bottom
df.loc["Class"] = class_values

# Step 4: Save the modified DataFrame to a new CSV file
df.to_csv(output_file)

print(f"Updated CSV file saved as '{output_file}'")


Updated CSV file saved as 'merged_GSE18520_fresh_with_Class.csv'
